# Introduction to ImageJ Ops

[ImageJ Ops](https://imagej.net/ImageJ_Ops) is a library for N-dimensional image processing.

The primary design goals of Ops are:

1. __Ease of use.__ Ops provides a wealth of easy-to-use image processing operations ("ops").
2. __Reusability.__ Ops extends Java's mantra of "[write once, run anywhere](https://en.wikipedia.org/wiki/Write_once,_run_anywhere)" to image processing algorithms. Algorithms written in the Ops framework are usable as-is from any [SciJava](https://imagej.net/SciJava)-compatible software project, such as [ImageJ](https://imagej.net/ImageJ), [CellProfiler](https://imagej.net/CellProfiler), [KNIME](https://imagej.net/KNIME), [OMERO](https://imagej.net/OMERO) and [Alida](https://imagej.net/Alida).
3. __Reproducibility.__ Ops are deterministic: calling the same op twice with the same arguments yields the same result, always. And all ops are versioned.
4. __Power.__ An op may consist of any number of typed input and output parameters. Ops may operate on arbitrary data structures, including images of N dimensions stored in a myriad of different ways: as files on disk, programmatically generated in memory, or in remote databases.
5. __Extensibility.__ Ops provides a robust framework for writing new ops, and for extending existing ops in new directions. See the "Extending ImageJ - Ops" tutorial notebook for details.
6. __Speed.__ The Ops framework provides a means to override any general-but-slow op with a faster-but-more-specific alternative, fully transparently to the user.

Let's dive in to doing some image processing using ImageJ Ops.

## Getting started

In [ ]:
// Behind a firewall? Configure your proxy settings here.
//System.setProperty("http.proxyHost","myproxy.domain")
//System.setProperty("http.proxyPort","8080")

//////////////////////////////////////////////////////////////
// Load ImageJ. This may take some minutes the first time   //
// while ImageJ is installed into ~/.groovy/grapes locally. //
//////////////////////////////////////////////////////////////
@GrabResolver(name='imagej', root='http://maven.imagej.net/content/groups/public/')
@Grab('net.imagej:imagej:2.0.0-rc-58')
import net.imagej.ImageJ
ij = new ImageJ()

// Define some handy shorthands!
show = { image ->
  ij.notebook().display(image)
}
import net.imglib2.RandomAccessibleInterval
tile = { images ->
  int[] gridLayout = images[0] in List ?
    [images[0].size, images.size] : // 2D images list
    [images.size] // 1D images list
  RandomAccessibleInterval[] rais = images.flatten()
  mosaic = ij.notebook().mosaic(gridLayout, rais)
  ij.notebook().display(mosaic)
}

println("ImageJ is ready to go.")

Let's open up the friendly Clown image to use for our experiments. (For the [coulrophobic](https://en.wikipedia.org/wiki/Coulrophobia), feel free to use the [Fluorescent Cells](http://imagej.net/images/FluorescentCells.jpg) instead.)

In [ ]:
clown = ij.io().open("http://imagej.net/images/clown.png")
show(clown)

He's a bit large, so let's scale him down. We'll use the `transform.scaleView` op.

For succinctness, we'll write only `scaleView` rather than the fully qualified op name `transform.scaleView`.

Right now, there is no default interpolation strategy, so we have to explicitly specify one. We'll use N-linear interpolation, since it tends to look pretty nice.

In [ ]:
import net.imglib2.interpolation.randomaccess.NLinearInterpolatorFactory

scaleFactors = [0.5, 0.5, 1] // Reduce X and Y to 50%; leave C dimension alone.
interpolationStrategy = new NLinearInterpolatorFactory()

image = ij.op().run("scaleView", clown, scaleFactors, interpolationStrategy)
show(image)

Lastly, let's prepare a grayscale version, which will work better for demonstrating certain ops later. We'll crop to only the red channel for that.

In [ ]:
import net.imglib2.FinalInterval
w = image.dimension(0); h = image.dimension(1)
slice = FinalInterval.createMinSize(0, 0, 0, w, h, 1)

grayImage = ij.op().run("crop", image, slice, true)
show(grayImage)

## Learning about available ops

You can get a complete list of available ops using the `ops()` method. But it is a bit overwhelming, so here is a structured version organized by namespace:

In [ ]:
import net.imagej.ops.OpUtils
opsByNS = [:]
ij.op().ops().each{op ->
  ns = OpUtils.getNamespace(op)
  name = OpUtils.stripNamespace(op)
  if (!opsByNS.containsKey(ns)) opsByNS.put(ns, name)
  else opsByNS.put(ns, opsByNS.get(ns) + ', ' + name)
}
opsByNS.put('<global>', opsByNS.remove(null))
opsByNS

There is a helpful `help` op which prints out information about an op. We can use it to better understand what sorts of operations are possible, and what kinds of arguments they expect:

In [ ]:
ij.op().help('gauss')

The information above tells us that there are two available `gauss` operations: one which takes a list (`double[]`) of sigmas, and another which takes a single (`double`) sigma.

There are a couple of important subtleties here:

1. Some arguments have a `?` suffix, which means they are optional. If you leave them off, something reasonable will be done by default. In this case, the `out` and `outOfBounds` arguments are optional. You can leave them off ___in right-to-left order___. E.g., in the case above, calling `gauss(a, b, c)` means `gauss(out, in, sigmas)` and _not_ `gauss(in, sigmas, outOfBounds)`. If you want to omit the `out` argument while passing the `outOfBounds` argument, you can pass `null` for `out`—i.e., `gauss(null, in, sigmas, outOfBounds)`.

2. The `out` argument is both an input _and_ an output. Hence, whatever object you pass as the `out` parameter will also be returned as the output. In this case, since `out` is optional, if you do not pass the `out` parameter (or you pass `null`), then one will be synthesized and returned.

Lastly, you might be wondering what a `RandomAccessibleInterval` is. For now, it is enough to know it is an ImageJ image. See the "Image samples" tutorial for a more detailed introduction.

Let's try calling the first `gauss` op:

In [ ]:
// Smudge him up horizontally.
double[] horizSigmas = [8, 0, 0]
horizGauss = ij.op().filter().gauss(image, horizSigmas)

// And now vertically.
double[] vertSigmas = [0, 8, 0]
vertGauss = ij.op().filter().gauss(image, vertSigmas)

// We can also blur the channels.
double[] channelSigmas = [0, 0, 1]
channelGauss = ij.op().filter().gauss(image, channelSigmas)

tile([image, horizGauss, vertGauss, channelGauss])

There are two main ways to execute an op:

1. The _type-safe_ way using a built-in method:
   ```java
   ij.op().foo().bar(...)
   ```
   This way tends to be nicer from type-safe languages like Java.
   
2. The _dynamic_ way using the `run` method:
   ```java
   ij.op().run("foo.bar", ...)
   ```
   This way tends to be nicer from dynamically typed scripting languages.

With the dynamic approach, passing the namespace is optional, so you can also write:
```java
ij.op().run("bar", ...)
```
If there are ops with the same name across multiple namespaces (e.g., `math.and` and `logic.and`), then passing the short name will consider the op signatures across all namespaces.

You will see both syntaxes used throughout this notebook.

## Generating images

Images have to come from somewhere. You can use `ij.io().open(...)` to read one in from an external source such as files on disk, like we did above. But there are other ways, too.

### Creating an empty image

You can create an empty (zero-filled) image using the `create.img` op.

In [ ]:
ij.op().help("create.img")

In [ ]:
def info(name, image) {
  imageType = image.firstElement().getClass().getSimpleName()
  println(name + " = " + image.dimension(0) + " x " + image.dimension(1) + " x " + image.dimension(2) + " : " + imageType)
}

info("Original ", image)

// Create an empty image of the same size as an existing image.
empty = ij.op().run("create.img", image)
info("Empty one", empty)

// Create an empty image based on another image, but of different type.
import net.imglib2.type.logic.BitType
bitType = ij.op().run("create.img", image, new BitType())
info("Bit image", bitType)

// Create an image from scratch.
import net.imglib2.type.numeric.real.FloatType
info("Small float", ij.op().run("create.img", [25, 17, 2], new FloatType()))

See "Converting between image types" below for more about image types.

### Copying an image

You can easily make a copy of an image using ops from the `copy` namespace.

In [ ]:
ij.op().help("copy.rai")

In [ ]:
copy = ij.op().run("copy.rai", image)
show(copy)

### Generating an image from a formula

The `equation` op allows you to generate an image from a formula in JavaScript syntax.

Such images can be useful for testing without needing an external image source, or a long and bulky list of numbers.

In [ ]:
ij.op().help("equation")

In [ ]:
import net.imglib2.type.numeric.integer.UnsignedByteType
sinusoid = ij.op().run("create.img", [150, 100], new UnsignedByteType())
formula = "63 * (Math.cos(0.3*p[0]) + Math.sin(0.3*p[1])) + 127"
show(ij.op().image().equation(sinusoid, formula))

### Wrapping an array as an image

There is no op yet to create an image by wrapping an existing array of values. For now, you can use ImgLib2 utility methods for that:

In [ ]:
// Here we have a gradient ramp in an array.
w = 160; h = 96
byte[] pixels = new byte[w * h]
for (y in 0..h-1) {
  for (x in 0..w-1) {
    pixels[y * w + x] = x + y
  }
}

// Wrap the array into an image.
import net.imglib2.img.array.ArrayImgs
ramp = ArrayImgs.unsignedBytes(pixels, w, h)
show(ramp)

## Computing statistics

The `stats` package has routines for computing numerical statistics.

Here is a rundown of many of them:

In [ ]:
sinusoid32 = ij.op().run("create.img", [150, 100])
formula = "63 * (Math.cos(0.3*p[0]) + Math.sin(0.3*p[1])) + 127"
ij.op().image().equation(sinusoid32, formula)

println("geometricMean = " + ij.op().stats().geometricMean(sinusoid32))
println("harmonicMean = " + ij.op().stats().harmonicMean(sinusoid32))
println("kurtosis = " + ij.op().stats().kurtosis(sinusoid32))
println("max = " + ij.op().stats().max(sinusoid32))
println("mean = " + ij.op().stats().mean(sinusoid32))
println("median = " + ij.op().stats().median(sinusoid32))
println("min = " + ij.op().stats().min(sinusoid32))
println("moment1AboutMean = " + ij.op().stats().moment1AboutMean(sinusoid32))
println("moment2AboutMean = " + ij.op().stats().moment2AboutMean(sinusoid32))
println("moment3AboutMean = " + ij.op().stats().moment3AboutMean(sinusoid32))
println("moment4AboutMean = " + ij.op().stats().moment4AboutMean(sinusoid32))
println("size = " + ij.op().stats().size(sinusoid32))
println("skewness = " + ij.op().stats().skewness(sinusoid32))
println("stdDev = " + ij.op().stats().stdDev(sinusoid32))
println("sum = " + ij.op().stats().sum(sinusoid32))
println("sumOfInverses = " + ij.op().stats().sumOfInverses(sinusoid32))
println("sumOfLogs = " + ij.op().stats().sumOfLogs(sinusoid32))
println("sumOfSquares = " + ij.op().stats().sumOfSquares(sinusoid32))
println("variance = " + ij.op().stats().variance(sinusoid32))

Note that `min` and `max` are particularly useful, since they report the actual minimum and maximum values of the image. These values are helpful for [autoscaling](https://imagej.net/Troubleshooting#The_image_I_loaded_is_displayed_all_black.21_But_it_is_not_black.21) to improve an image's appearance.

## Math on images

In the `math` namespace, Ops provides traditional mathematical operations such as `add`, `subtract`, `multiply` and `divide`. These operations are overloaded in several ways:

* Operate pixelwise between two images—e.g., `math.add(image1, image2)` when `image1` and `image2` have the same dimensions.
* Operate between an image and a constant—e.g., `math.add(image, 5)` to add 5 to each sample of `image`.
* Operate between two numerical values—e.g., `math.add(3, 5)` to compute the sum of 3 and 5.

Some `math` ops are also already heavily optimized, since we used the `math.add` op as a testbed to validate that Ops could perform as well or better than ImageJ 1.x does.

In [ ]:
// Prepare a couple of equally sized images.
import net.imglib2.type.numeric.real.FloatType
image1 = ij.op().run("create.img", [160, 96], new FloatType())
image2 = ij.op().run("copy.rai", image1)

// Gradient toward bottom right.
ij.op().image().equation(image1, "p[0] + p[1]")
minMax1 = ij.op().stats().minMax(image1)
println("image1 range = (" + minMax1.getA() + ", " + minMax1.getB() + ")")

// Sinusoid.
ij.op().image().equation(image2, "64 * (Math.sin(0.1 * p[0]) + Math.cos(0.1 * p[1])) + 128")
minMax2 = ij.op().stats().minMax(image2)
println("image2 range = (" + minMax2.getA() + ", " + minMax2.getB() + ")")

tile([image1, image2])

Let's test `math.add(image, number)`:

In [ ]:
addImage = image1 // Try also with image2!
tile([
  addImage,
  ij.op().run("math.add", ij.op().run("copy.rai", addImage), 60),
  ij.op().run("math.add", ij.op().run("copy.rai", addImage), 120),
  ij.op().run("math.add", ij.op().run("copy.rai", addImage), 180)
])

Notice how we had to make a copy of the source image for each `add(image, number)` above? This is because right now, the best-matching `math.add` op is an _inplace_ operation, modifying the source image. Ops is still young, and needs more fine tuning! In the meantime, watch out for details like this.

Now we'll try `math.add(image1, image2)` and `math.subtract(image1, image2)`:

In [ ]:
sum = ij.op().run("math.add", image1, image2)
diff = ij.op().run("math.subtract", image1, image2)
tile([sum, diff])

Here is `math.multiply(image1, image2)`:

In [ ]:
show(ij.op().run("math.multiply", image1, image2))

And finally `math.divide(image1, image2)`:

In [ ]:
show(ij.op().run("math.divide", image1, image2))

## Evaluating expressions

ImageJ Ops offers a powerful expression evaluation op, built on SciJava's [Parsington](https://github.com/scijava/parsington) library:

In [ ]:
ij.op().help("eval")

Operators in the expression map to ops as follows:

In [ ]:
import net.imagej.ops.eval.OpEvaluator
// FIXME: Use `new OpEvaluator().getOpMap()` once imagej-ops 0.36.0 is released.
opMapField = OpEvaluator.class.getDeclaredField("opMap")
opMapField.setAccessible(true)
opMap = opMapField.get(new OpEvaluator())

You can also call any op in an `eval` statement as a function, using familiar function syntax.

Here is an example of the `eval` op being used to compute a [Difference of Gaussians](https://en.wikipedia.org/wiki/Difference_of_Gaussians):

In [ ]:
image32 = ij.op().convert().float32(grayImage)
dogFormula = "gauss(image, [10, 10, 1]) - gauss(image, [5, 5, 1])"
dog = ij.op().eval(dogFormula, ["image": image32])
show(dog)

## Converting between image types

Did you notice in the previous section that we computed the DoG on a `float32` version of the image? If we do not do this, the result will be wrong:

In [ ]:
dogWrong = ij.op().eval(dogFormula, ["image": grayImage])
dogConverted = ij.op().convert().uint8(dog)
tile([dogWrong, dogConverted])

As you can see, a DoG computed on the original `uint8` version of the image looks essentially the same as the `float32` DoG converted to `uint8`.

The issue is that `uint8` images have only 8 bits per channel, with values ranging from 0 to 255. So while the equation is dutifully evaluated, non-integer values are truncated, and then only the lowest 8 bits are retained.

Using a richer image type like `float32` avoids the problem. The `float32` type uses 32-bit [IEEE floating point](https://en.wikipedia.org/wiki/IEEE_floating_point) numbers for its samples, which an approximate range of
$-3.4 \times 10^{38}$
to
$3.4 \times 10^{38}$.
This type requires four times as much memory as `uint8`, but math errors will accumulate much less severely in common cases.

<div style="float: left"><img src="http://imagej.net/_images/a/a8/Imagej1-icon.png" width="48"></div>

For those familiar with [ImageJ 1.x](http://imagej.net/ImageJ1), `float32` corresponds to ImageJ 1.x's "32-bit" type in the Image &rarr; Type menu, and is typically what people choose when performing sequences of mathematical operations.

<div style="clear: left"></div>

### Built-in image types

In ImageJ Ops, many image types are available:

In [ ]:
ij.op().ops().findAll{op ->
  op.startsWith("convert.big") ||
  op.startsWith("convert.bit") ||
  op.startsWith("convert.cfloat") ||
  op.startsWith("convert.float") ||
  op.startsWith("convert.int") ||
  op.startsWith("convert.uint")
}.collect{op -> op[8..-1]}

In some circumstances, one of the above types is more appropriate than `float32`. For example, if we are squaring an image, we could use `uint16`, since the square of a number from 0 to 255 will always be in the range of 0 to 65535. It may sometimes make sense to use a high-precision integer type such as `uint128` rather than a potentially lossy floating point type like `float32`.

### Methods of type conversion

Ops provides four built-in ways of converting image values across types:

* `convert.copy` tranfers the values directly. This is the fastest method, but there is no bounds checking, so you may see overflow, or in some cases even end up with invalid sample values.
* `convert.clip` is like `copy` but includes bounds checking. Values less than the minimum are _clipped_ (also called _clamped_) to the minimum value, and values greater than the maximum are clipped to the maximum.
* `convert.scale` multiplies sample values by a scale factor, such that the intensities fall into the same _relative histogram_ over the whole target type:
  $[sourceTypeMin, sourceTypeMax] \to [destTypeMin, destTypeMax]$.
  E.g., converting from `uint8` to `uint16` linearly scales
  $[0, 255] \to [0, 65535]$.
* `convert.normalizeScale` multiplies sample values by a scale factor, such that intensities are distributed across the _full range_ of the target type:
  $[sourceDataMin, sourceDataMax] \to [destTypeMin, destTypeMax]$.
  E.g., converting a `uint8` image with an actual minimum sample value of 16 and actual maximum sample value of 127 to `uint16` linearly scales
  $[16, 127] \to [0, 65535]$.

Below, we show how to convert images using these approaches.

In [ ]:
// Make a handy method for printing the image min/max.
printMinMax = { prefix, image ->
  minMax = ij.op().stats().minMax(image)
  println(prefix + ": min = " + minMax.getA() + ", max = " + minMax.getB())
}

// Create an image with an illustrative range of intensities.
import net.imglib2.type.numeric.integer.UnsignedShortType
imageToConvert = ij.op().run("create.img", [96, 64], new UnsignedShortType())
formula = "128 * (Math.cos(0.3*p[0]) + Math.sin(0.3*p[1])) + 384"
ij.op().image().equation(imageToConvert, formula)
printMinMax("Original", imageToConvert)
show(imageToConvert)

Now we will call the various convert ops. Please be aware that these ops currently have architectural issues, which we plan to address in a future version of ImageJ Ops. The code below works, but is subject to change until the ImageJ Ops 1.0.0 release.

In [ ]:
import net.imagej.ops.Ops
import net.imagej.ops.special.computer.Computers
import net.imglib2.type.numeric.integer.UnsignedByteType

convertMethods = [
  Ops.Convert.Copy.class,
  Ops.Convert.Clip.class,
  // HACK: convert.scale op is disabled here due to a bug in Ops.
//  Ops.Convert.Scale.class,
  Ops.Convert.NormalizeScale.class
]

convertedImages = []
for (convertMethod in convertMethods) {
  // Create a uint8 destination image for the conversion.
  convertedImage = ij.op().run("create.img", imageToConvert, new UnsignedByteType())

  // Look up the needed convert op for converting from source to destination.
  inType = imageToConvert.firstElement() // Type from which we are converting.
  outType = convertedImage.firstElement() // Type to which we are converting.
  convertOp = Computers.unary(ij.op(), convertMethod, outType, inType)
  
  // NB: Prepare the convert op to behave properly.
  // The need for these calls is hacky, and will be fixed in a future version of Ops.
  convertOp.checkInput(inType, outType)
  convertOp.checkInput(imageToConvert)

  // Run this convert op on every sample of our source image.
  ij.op().run("map", convertedImage, imageToConvert, convertOp)
  methodName = convertMethod.getField("NAME").get(null)
  printMinMax(methodName, convertedImage)
  convertedImages.add(convertedImage)
}

tile(convertedImages)

The code above makes use of the `map` op to execute the appropriate `convert` op on each element of a collection. 

## Filtering

There are lots of image filtering operations available in the `filter` namespace:

In [ ]:
ij.op().ops().findAll{op ->
  op.startsWith("filter.")
}.collect{op -> op[7..-1]}

Here is a demonstration of some of them:

In [ ]:
ij.op().help("filter.addPoissonNoise")

In [ ]:
imageToFilter = image32
radius = 3

// We will use a 3x3 diamond as our neighborhood here.
import net.imglib2.algorithm.neighborhood.DiamondShape
shape = new DiamondShape(radius)

// Add Poisson noise.
addPoissonNoise = ij.op().run("create.img", imageToFilter)
ij.op().filter().addPoissonNoise(addPoissonNoise, imageToFilter)

// Gaussian blur.
gauss = ij.op().filter().gauss(imageToFilter, radius)

// Median filter.
median = ij.op().run("create.img", imageToFilter)
ij.op().filter().median(median, imageToFilter, shape)

// Min filter.
min = ij.op().run("create.img", imageToFilter)
ij.op().filter().min(min, imageToFilter, shape)

// Max filter.
max = ij.op().run("create.img", imageToFilter)
ij.op().filter().max(max, imageToFilter, shape)

// Display the results side by side.
tile([
  [imageToFilter, addPoissonNoise, gauss],
  [median, min, max]
])

Finally, let's take a special look at the Difference of Gaussians op, `filter.dog`:

In [ ]:
imageToProcess = image32
sigma1 = 5
sigma2 = 10

// Difference of Gaussians.
dog = ij.op().filter().dog(imageToProcess, sigma1, sigma2) // gauss(sigma2) - gauss(sigma1)

// We can also use eval to perform the DoG.
vars = [
  "i": imageToProcess,
  "sigma1": [sigma1, sigma1],
  "sigma2": [sigma2, sigma2]
]
evalDoG = ij.op().eval("gauss(i, sigma2) - gauss(i, sigma1)", vars)

tile([dog, evalDoG])

## Transforming images

Image transformations such as rotation, scaling and cropping are accomplished using ops of the `transform` namespace.

Most ops of this namespace have the nice property of being _views_: they do not actually copy image samples, but rather wrap the image, offering a modified "window" into the original data. Using views helps to greatly reduce computer memory usage, at a very minimal time performance cost. If you need a deep copy of the image for some reason (e.g., if time performance is paramount, or if you want to modify the transformed image samples in-place without affecting other transformed versions of the image), you can still copy it using the `copy` namespace; see "Generating images" above for details.

In [ ]:
ij.op().ops().findAll{op ->
  op.startsWith("transform.")
}.collect{op -> op[10..-1]}

### Rotating an image

The `transform.rotateView` op rotates the image 90 degrees from one dimensional axis to another. E.g., in the 2D case, rotating Y &rarr; X is a clockwise rotation, while rotating X &rarr; Y is a counterclockwise one.

In [ ]:
ij.op().help("rotateView")

Here is an example usage of the `transform.rotateView` op.

Note that at the moment, for rather technical reasons, rotating an image removes the bounding box; for the rotated result to be fully usable afterward, a new bounding box must be specified. If you want to understand this, then study the following code carefully to see how this is done using the `transform.intervalView` op, which adds or changes the bounding box of an image. Otherwise, you may simply copy the `rotate` function below and enjoy. The `rotateView` op in a future version of ImageJ Ops will probably rebound the image automatically, to avoid this complication.

In [ ]:
def rotate(image, fromAxis, toAxis) {
  // Perform the rotation.
  rotated = ij.op().run("rotateView", image, toAxis, fromAxis)
  
  // Adjust the bounds to match the rotated interval.
  // Some day, the rotate op will take care of this automatically.
  min = new long[image.numDimensions()]
  max = new long[image.numDimensions()]
  image.min(min)
  image.max(max)
  minFrom = min[toAxis]; minTo = -max[fromAxis]
  maxFrom = max[toAxis]; maxTo = -min[fromAxis]
  min[toAxis] = minTo; min[fromAxis] = minFrom
  max[toAxis] = maxTo; max[fromAxis] = maxFrom
  return ij.op().run("intervalView", rotated, min, max)
}

bounds = {interval ->
  return "(" + interval.min(0) + ", " + interval.min(1) + ") - " +
         "(" + interval.max(0) + ", " + interval.max(1) + ")"
}

// Define our axis indices, for readability.
// In this case, we have a priori knowledge.
x = 0; y = 1; c = 2

// Rotate the image.
rotated = rotate(image, y, x) // 90 degrees clockwise
//rotated = rotate(image, x, y) // 90 degrees counter-clockwise
//rotated = rotate(image, x, c) // rotate through channels! WAT

// The interval bounds have changed!
println("Old bounds: " + bounds(image))
println("New bounds: " + bounds(rotated))

show(rotated)

### Cropping an image

The `transform.crop` op crops an image N-dimensionally. E.g., you can use it to create a substack of a 3D dataset, cut out irrelevant channels, or crop the XY planes as with 2D image processing software.

In [ ]:
ij.op().help("crop")

Below, we show two ways to crop an image: 1) with `transform.crop`; and 2) using `transform.intervalView`. The former translates the image back to the origin, while the latter does not.

In [ ]:
import net.imglib2.FinalInterval
region = FinalInterval.createMinSize(75, 27, 0, 40, 28, 3)

eye = ij.op().run("crop", image, region)
eyeView = ij.op().run("intervalView", image, region)

tile([eye, eyeView])

### Scaling an image

To perform [image scaling](https://en.wikipedia.org/wiki/Image_scaling), use the `transform.scaleView` op. You already saw it in action in the "Getting started" section, but here it is again—this time enlarging an image rather than reducing it.

Just for fun, we compare three different interpolation strategies: [nearest neighbor](https://en.wikipedia.org/wiki/Nearest-neighbor_interpolation), [N-linear](https://en.wikipedia.org/wiki/Linear_interpolation), and [Lanczos](https://en.wikipedia.org/wiki/Lanczos_resampling).

In [ ]:
import net.imglib2.interpolation.randomaccess.NearestNeighborInterpolatorFactory
import net.imglib2.interpolation.randomaccess.NLinearInterpolatorFactory
import net.imglib2.interpolation.randomaccess.LanczosInterpolatorFactory

scaleFactors = [4, 4, 1] // Enlarge X and Y by 4x; leave channel count the same.

nearestNeighborEye = ij.op().run("scaleView", eye, scaleFactors, new NearestNeighborInterpolatorFactory())
nLinearEye = ij.op().run("scaleView", eye, scaleFactors, new NLinearInterpolatorFactory())
lanczosEye = ij.op().run("scaleView", eye, scaleFactors, new LanczosInterpolatorFactory())

tile([nearestNeighborEye, nLinearEye, lanczosEye])

Of course, some detail from the original image has been lost, since we scaled down and then back up again.

### Padding an image

The `transform.intervalView` can also be used to expand the bounding box of an image. However, there is one catch: you must first decide what the out-of-bounds sample values will be. The `transform.extend` ops achieve this goal. If you forget to extend the image before padding it via `intervalView`, you will receive an exception when attempting to query any out-of-bounds samples.

Note that like `transform.rotateView` the various `transform.extend` ops explicitly remove the bounding box of an image, expanding the defined sample values to infinity in all directions. In most circumstances, you will want to use `transform.intervalView` to rebound the image after extending it.

In [ ]:
ij.op().ops().findAll{op ->
  op.startsWith("transform.extend")
}.collect{op -> op[10..-1]}

Here are some side-by-side examples of what happens when you pad an image with these various approaches:

In [ ]:
def pad(image, extended, t, r, b, l) {
  min = new long[image.numDimensions()]
  max = new long[image.numDimensions()]
  image.min(min)
  image.max(max)
  min[0] -= l; min[1] -= t; max[0] += r; max[1] += b
  return ij.op().run("intervalView", extended, min, max)
}

// Define the top, right, bottom and left padding amounts.
t = r = b = l = 20

// Pad the image with different out-of-bounds strategies.

extendedBorder = ij.op().run("transform.extendBorderView", eye)
paddedBorder = pad(eye, extendedBorder, t, r, b, l)

extendedMirrorDouble = ij.op().run("transform.extendMirrorDoubleView", eye)
paddedMirrorDouble = pad(eye, extendedMirrorDouble, t, r, b, l)

extendedMirrorSingle = ij.op().run("transform.extendMirrorSingleView", eye)
paddedMirrorSingle = pad(eye, extendedMirrorSingle, t, r, b, l)

extendedPeriodic = ij.op().run("transform.extendPeriodicView", eye)
paddedPeriodic = pad(eye, extendedPeriodic, t, r, b, l)

minValue = eye.firstElement().getMinValue().doubleValue()
maxValue = eye.firstElement().getMaxValue().doubleValue()
extendedRandom = ij.op().run("transform.extendRandomView", eye, minValue, maxValue)
paddedRandom = pad(eye, extendedRandom, t, r, b, l)

value = eye.firstElement().copy(); value.set(100)
extendedValue = ij.op().run("transform.extendValueView", eye, value)
paddedValue = pad(eye, extendedValue, t, r, b, l)

extendedZero = ij.op().run("transform.extendZeroView", eye)
paddedZero = pad(eye, extendedZero, t, r, b, l)

show(paddedRandom)
/*
tile([
  [paddedBorder, paddedMirrorDouble, paddedMirrorSingle, paddedPeriodic],
  [paddedRandom, paddedValue, paddedZero]
])
*/

## Thresholding

Ops has all the same global thresholding methods as ImageJ 1.x, as well as some local thresholding methods.

In [ ]:
ij.op().ops().findAll{op ->
  op.startsWith("threshold.")
}.collect{op -> op[10..-1]}

Let's check out the global thresholding algorithms:

In [ ]:
threshImage = grayImage

tHuang = ij.op().threshold().huang(threshImage)
tIJ1 = ij.op().threshold().ij1(threshImage) // ImageJ 1.x calls this "Default"
tIntermodes = ij.op().threshold().intermodes(threshImage)
tIsoData = ij.op().threshold().isoData(threshImage)
tLi = ij.op().threshold().li(threshImage)
tMaxEntropy = ij.op().threshold().maxEntropy(threshImage)
tMaxLikelihood = ij.op().threshold().maxLikelihood(threshImage)
tMean = ij.op().threshold().mean(threshImage)
tMinError = ij.op().threshold().minError(threshImage)
tMinimum = ij.op().threshold().minimum(threshImage)
tMoments = ij.op().threshold().moments(threshImage)
tOtsu = ij.op().threshold().otsu(threshImage)
tPercentile = ij.op().threshold().percentile(threshImage)
tRenyiEntropy = ij.op().threshold().renyiEntropy(threshImage)
tRosin = ij.op().threshold().rosin(threshImage)
tShanbhag = ij.op().threshold().shanbhag(threshImage)
tTriangle = ij.op().threshold().triangle(threshImage)
tYen = ij.op().threshold().yen(threshImage)

tile([
  [        tHuang,          tIJ1, tIntermodes,  tIsoData,       tLi, tMaxEntropy],
  [tMaxLikelihood,         tMean,   tMinError,  tMinimum,  tMoments,       tOtsu],
  [   tPercentile, tRenyiEntropy,      tRosin, tShanbhag, tTriangle,        tYen]
])

Let's try some local thresholding. This is a bit more complex, because local thresholding algorithms typically need a neighborhood, and often some other secondary parameters. We will take the Bernsen algorithm for a spin:

In [ ]:
ij.op().help("threshold.localBernsenThreshold")

In [ ]:
import net.imglib2.algorithm.neighborhood.HyperSphereShape

threshImage = grayImage

// Secondary parameter values. These are wild guesses.
contrastThreshold = 10
halfMaxValue = 10

// Let's try with a variety of neighborhood radii and compare side-by-side.
import net.imglib2.type.logic.BitType
radii = [1, 3, 5, 8, 12, 15]
bernsenImages = []
for (radius in radii) {
  binaryImage = ij.op().run("create.img", threshImage, new BitType())
  ij.op().threshold().localBernsenThreshold(binaryImage, threshImage, new HyperSphereShape(radius), contrastThreshold, halfMaxValue)
  bernsenImages.add(binaryImage)
}
tile(bernsenImages)

Nice illustration of how much better local thresholding can be, eh? The limited scope when thresholding each pixel can go a long way toward correcting for issues like uneven illumination.

## Mathematical morphology

Ops includes operators for [mathematical morphology](https://en.wikipedia.org/wiki/Mathematical_morphology) in both binary and grayscale. The two most basic morphological operators are:

* _erosion_ – reducing bright pixels at the borders
* _dilation_ - growing bright pixels at the borders

The next two most basic are:

* _opening_ - dilation of the erosion
* _closing_ - erosion of the dilation

These latter two are not strictly necessary, since you can simply call `erode` followed by `dilate` or vice versa, but they are convenient as a shorthand.

In [ ]:
// HINT: Try different binary images here!
morphImage = binaryImage

// We will use the smallest radius: a single pixel.
import net.imglib2.algorithm.neighborhood.HyperSphereShape
shape = new HyperSphereShape(1)

erode = ij.op().morphology().erode(morphImage, shape)
dilate = ij.op().morphology().dilate(morphImage, shape)
open = ij.op().morphology().open(morphImage, [shape])
close = ij.op().morphology().close(morphImage, [shape])
// Let's also check whether open and close visually match what we expect.
manualOpen = ij.op().morphology().dilate(erode, shape)
manualClose = ij.op().morphology().erode(dilate, shape)

tile([[erode, dilate], [open, close], [manualOpen, manualClose]])

The open and close look good—they match their respective definitions.

Let's quickly try some [grayscale morphological operations](https://en.wikipedia.org/wiki/Mathematical_morphology#Grayscale_morphology) as well. This time we also invoke the [top-hat transform](https://en.wikipedia.org/wiki/Top-hat_transform).

In [ ]:
morphImage = grayImage

// We will use a larger radius of 3 for our neighborhood this time.
import net.imglib2.algorithm.neighborhood.HyperSphereShape
shape = new HyperSphereShape(3)

erode = ij.op().morphology().erode(morphImage, shape)
dilate = ij.op().morphology().dilate(morphImage, shape)
open = ij.op().morphology().open(morphImage, [shape])
close = ij.op().morphology().close(morphImage, [shape])
blackTopHat = ij.op().morphology().blackTopHat(morphImage, [shape])
whiteTopHat = ij.op().morphology().topHat(morphImage, [shape])

tile([[erode, dilate], [open, close], [blackTopHat, whiteTopHat]])

## Nerd digression

Congratulations on making it this far! Here is a cookie for you:

In [ ]:
import net.imglib2.interpolation.randomaccess.NLinearInterpolatorFactory
wideEye = ij.op().run("scaleView", eye, [2, 1, 1], new NLinearInterpolatorFactory())
ij.op().run("ascii", wideEye)

So classic!

You _did_ run section 9.2 "Cropping an image" first, right?